In [17]:
import re
from itertools import product
from string import ascii_letters

In [18]:
def all_strings(n):
    for s in product(ascii_letters, repeat=n):
        yield ''.join(s)

In [19]:
pattern = re.compile("o[gh]")

In [25]:
print(pattern.fullmatch("og"))
print(pattern.fullmatch("dog"))
print(pattern.fullmatch("ogre"))
print(pattern.fullmatch("doggie", 1, 3))

<_sre.SRE_Match object; span=(0, 2), match='og'>
None
None
<_sre.SRE_Match object; span=(1, 3), match='og'>


In [26]:
match_2 = [string for string in all_strings(2) if pattern.fullmatch(string)]
match_2

['og', 'oh']